In [ ]:
!git clone https://github.com/mszczesniak02/bachlor_google

Cloning into 'bachlor_google'...
remote: Enumerating objects: 1133, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 1133 (delta 8), reused 12 (delta 3), pack-reused 1106 (from 1)
Receiving objects: 100% (1133/1133), 66.92 MiB | 19.62 MiB/s, done.
Resolving deltas: 100% (245/245), done.


In [ ]:
!cp -r /content/bachlor_google/DeepCrack/ .

In [ ]:
!pip install segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.9 MB/s eta 0:00:00


In [ ]:
import albumentations as A                              # for augmentation transform

import numpy as np                                      # sci kit specials ;D
import matplotlib.pyplot as plt                         # plots
from PIL import Image                                   # for opening images as numpy arrays or torch tensors

from datetime import datetime                           # for model timestamp

import torch

from torch.utils.data import Dataset                    # preset class for creating a dataset
from torch.utils.data import random_split               # for splitting datasets into training, test, validation
from torch.utils.data import DataLoader                 # self-explanitory
import segmentation_models_pytorch as smp               # preset model

from tqdm import tqdm                                   # for the progress bar
import os                                               # for accessing files and setting proper paths to   them

from torch.utils.tensorboard import SummaryWriter       # tensorboard srv
from torch.nn.functional import binary_cross_entropy
from torchvision.ops import sigmoid_focal_loss


/home/krzeslaav/bachlor/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-26 21:49:21.190863: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
DEBUG = True

if DEBUG==True:

  MASK_TRAIN_PATH = "../assets/datasets/DeepCrack/train_lab"
  IMG_TRAIN_PATH = "../assets/datasets/DeepCrack/train_img"
  MASK_TEST_PATH = "../assets/datasets/DeepCrack/test_lab"
  IMG_TEST_PATH = "../assets/datasets/DeepCrack/test_img"
  DEVICE = "cuda"
  BATCH_SIZE = 2
  WORKERS = 2

else:
  MASK_TRAIN_PATH = "/content/DeepCrack/train_lab"
  IMG_TRAIN_PATH = "/content/DeepCrack/train_img"
  MASK_TEST_PATH = "/content/DeepCrack/test_lab"
  IMG_TEST_PATH = "/content/DeepCrack/test_img"
  BATCH_SIZE = 32


  DEVICE="cuda"
  WORKERS = 2

PIN_MEMORY = True
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
EPOCHS = 40

EARLY_STOPPING_PATIENCE = 15

SCHEDULER_PATIENCE = 5
SCHEDULER_FACTOR = 0.5

In [ ]:
class DeepCrackDataset(Dataset):
  def __init__(self, img_dir, mask_dir, transform=None):

    self.img_dir = img_dir
    self.mask_dir = mask_dir
    self.transform = transform

    # sort values so the file names corespoding to each other are loaded in order
    self.images = sorted([os.path.join(img_dir, file) for file in os.listdir(img_dir)] )
    self.masks = sorted([os.path.join(mask_dir, file) for file in os.listdir(mask_dir)])

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):
    np_image = np.array(Image.open(self.images[index]))
    np_mask = np.array(Image.open(self.masks[index]))


    if len(np_mask.shape) == 3:
      np_mask = np_mask[:,:,0]

    np_mask = (np_mask > 127).astype(np.uint8)

    if self.transform: # if using transforms
      t = self.transform(image=np_image, mask=np_mask)
      np_image = t["image"]
      np_mask = t["mask"]

    # conversion from numpy array convention to tensor via permute,
    #     then normalizing to [0,1] range, same for mask, only using binary data
    tensor_image = torch.from_numpy(np_image).permute(2, 0, 1).float() / 255.0
    tensor_mask = torch.from_numpy(np_mask).unsqueeze(0).float()

    return tensor_image,tensor_mask

In [ ]:
transform_train = A.Compose([
    A.Resize(512, 512),  
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.Rotate(limit=10, p=0.3),  
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3), 
])

transform_val = A.Compose([A.Resize(512,512)])

In [ ]:
class BCrossEntropyLoss(torch.nn.Module):
  def __init__(self, smooth=1e-16): # zmiana
    super(BCrossEntropyLoss,self).__init__()
    self.smooth = smooth
  def forward(self, predictions, targets):
    predictions = torch.sigmoid(predictions)
    loss = binary_cross_entropy(predictions, targets)
    return loss

In [ ]:
class FocalLoss(torch.nn.Module):
  def __init__(self, smooth=1e-16): # zmiana
    super(BCrossEntropyLoss,self).__init__()
    self.smooth = smooth
  def forward(self, predictions, targets):
    loss =sigmoid_focal_loss(predictions,targets)
    return loss

In [ ]:
class BCE_with_FocalLoss(torch.nn.Module):
  def __init__(self, smooth=1e-16): # zmiana
    super(BCE_with_FocalLoss,self).__init__()
    self.smooth = smooth
  def forward(self, predictions, targets):
    # loss_focal = sigmoid_focal_loss(predictions,targets)
    # loss_bce = binary_cross_entropy(predictions, targets)
    loss = 0.5 * sigmoid_focal_loss(predictions,targets)+ 0.5 * binary_cross_entropy(predictions, targets)

    return loss

In [ ]:
class DiceLoss(torch.nn.Module):
  def __init__(self, smooth=1e-6):
    super(DiceLoss,self).__init__()
    self.smooth = smooth
  def forward(self, predictions, targets):
    predictions = torch.sigmoid(predictions)

    predictions = predictions.view(-1)
    targets = targets.view(-1)

    intersection = (predictions * targets).sum()
    dice = (2. * intersection  + self.smooth) / (predictions.sum() + targets.sum() + self.smooth)

    return 1-dice

In [ ]:
class DiceFocalLoss(torch.nn.Module):
  def __init__(self, smooth=1e-6):
    super(DiceLoss,self).__init__()
    self.smooth = smooth
  def forward(self, predictions, targets):
    
    focal_loss = sigmoid_focal_loss(predictions,targets)

    predictions = torch.sigmoid(predictions)
    predictions = predictions.view(-1)
    targets = targets.view(-1)

    intersection = (predictions * targets).sum()
    dice = (2. * intersection  + self.smooth) / (predictions.sum() + targets.sum() + self.smooth)
    
    dice_loss = 1-dice

    the_loss = dice_loss * 0.5 + focal_loss * 0.5

    return the_loss

In [25]:
def calculate_metrics(predictions, targets, threshold=0.5):

    # Binaryzacja
    preds = (predictions > threshold).float()
    targets = targets.float()

    # Flatten
    preds_flat = preds.view(-1)
    targets_flat = targets.view(-1)

    # True/False Positives/Negatives
    TP = ((preds_flat == 1) & (targets_flat == 1)).sum().float()
    TN = ((preds_flat == 0) & (targets_flat == 0)).sum().float()
    FP = ((preds_flat == 1) & (targets_flat == 0)).sum().float()
    FN = ((preds_flat == 0) & (targets_flat == 1)).sum().float()

    conf_table = [[TP,FP],[FN, TN]]

    # Metryki
    epsilon = 1e-7  # Unikaj dzielenia przez zero

    accuracy = (TP + TN) / (TP + TN + FP + FN + epsilon)
    precision = TP / (TP + FP + epsilon)
    recall = TP / (TP + FN + epsilon)
    f1_score = 2 * (precision * recall) / (precision + recall + epsilon)
    specificity = TN / (TN + FP + epsilon)

    # IoU (Intersection over Union) - NAJWAŻNIEJSZA dla segmentacji!
    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum() - intersection
    iou = intersection / (union + epsilon)

    # Dice Coefficient
    dice = (2 * intersection) / (preds.sum() + targets.sum() + epsilon)

    return {
        'accuracy': accuracy.item(),
        'precision': precision.item(),
        'recall': recall.item(),
        'f1_score': f1_score.item(),
        'specificity': specificity.item(),
        'iou': iou.item(),
        'dice': dice.item(),
        'confusion_table': conf_table
    }

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):

    model.train()
    running_loss = .0
    metrics = {
        'iou': [], 'dice': [], 'recall': [],
        'precision': [], 'f1_score': []
    }

    loop = tqdm(train_loader, desc="Training", leave=False)
    for batch_idx, (images, masks) in enumerate(loop):
        images = images.to(device)
        masks = masks.to(device)
        # Forward pass
        predictions = model(images)
        loss = criterion(predictions, masks)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        with torch.no_grad():
            predictions_sigmoid = torch.sigmoid(predictions)
            batch_metrics = calculate_metrics(predictions_sigmoid, masks)

            for key in metrics.keys():
                value = batch_metrics[key]
                if isinstance(value, torch.Tensor):
                    value = value.cpu().item()
                metrics[key].append(batch_metrics[key])

        loop.set_postfix({'loss': loss.item()})

    loop.close()

    avg_loss = running_loss / len(train_loader)
    avg_metrics = {k: np.mean(v) for k, v in metrics.items()}
    avg_metrics['loss'] = avg_loss


    return avg_metrics

def validate(model, val_loader, criterion,device):
  model.eval()
  running_loss = 0.0
  metrics = {
        'iou': [], 'dice': [], 'recall': [],
        'precision': [], 'f1_score': []
    }
  with torch.no_grad():
    for images,masks in tqdm(val_loader, desc="Validation", leave=False):
      
      images = images.to(device)
      masks = masks.to(device)

      predictions = model(images)
      loss = criterion(predictions, masks)

      running_loss += loss.item()

      predictions_sigmoid = torch.sigmoid(predictions)
      batch_metrics = calculate_metrics(predictions_sigmoid, masks)

      for key in metrics.keys():
          value = batch_metrics[key]

          if isinstance(value, torch.Tensor):
              value = value.cpu().item()

          metrics[key].append(batch_metrics[key])

    avg_loss = running_loss / len(val_loader)
    avg_metrics = {k: np.mean(v) for k, v in metrics.items()}
    avg_metrics['loss'] = avg_loss

    return avg_metrics


In [ ]:
def main()-> int:

    its_training_time = datetime.now().strftime('H%M') # Jared Leto likes it
    writer = SummaryWriter(f"runs/model_{its_training_time}")

    train_set = DeepCrackDataset(IMG_TRAIN_PATH, MASK_TRAIN_PATH, transform_train=transform_train)
    val_dataset = DeepCrackDataset(IMG_TEST_PATH, MASK_TEST_PATH, transform_train=transform_val)

    test_set, val_set =  random_split(val_dataset, [.2, .8])

    train_loader    = DataLoader( train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=WORKERS, pin_memory=PIN_MEMORY)
    test_loader     = DataLoader( test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=WORKERS, pin_memory=PIN_MEMORY)
    val_loader      = DataLoader( val_set, batch_size=BATCH_SIZE , shuffle=False, num_workers=WORKERS    , pin_memory=PIN_MEMORY)

    print("Datasets sizes: ")
    print(f"\t  Train: {len(train_set)}")
    print(f"\t    Val: {len(val_set)}")
    print(f"\t   Test: {len(test_set)}")

    device = torch.device(DEVICE if torch.cuda.is_available() else "cpu")

    model = smp.Unet(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
    )

    model = model.to(device)

    criterion = DiceFocalLoss()

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
    )

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='max',  # Maksymalizuj IoU
        factor=SCHEDULER_FACTOR,
        patience=SCHEDULER_PATIENCE,
        min_lr=1e-7
    )

    print(f"\nTraining configuration:")
    print(f"\t       Optimizer: Adam")
    print(f"\t   Learning rate: {LEARNING_RATE}")
    print(f"\t    Weight decay: {WEIGHT_DECAY}")
    print(f"\t       Scheduler: ReduceLROnPlateau (patience={SCHEDULER_PATIENCE})")
    print(f"\t  Early stopping: patience={EARLY_STOPPING_PATIENCE}")
    print(f"\t          Epochs: {EPOCHS}")

    epochs = EPOCHS
    best_val_iou = 0.0
    best_epoch = 0
    patience_counter = 0

    writer.add_text("Hparams",f"""
    -           Learning Rate: {LEARNING_RATE}
    -              Batch Size: {BATCH_SIZE}
    -            Weight Decay: {WEIGHT_DECAY}
    -                  Epochs: {EPOCHS}
    -      Scheduler Patience: {SCHEDULER_PATIENCE}
    - Early Stopping Patience: {EARLY_STOPPING_PATIENCE}
    -                  Device: {DEVICE}
""")



    print(f"Epochs:{epochs}, current:",end='')
    for epoch in range(epochs):
        print(f"{epoch},")

        train_metrics = train_epoch(model, train_loader, criterion, optimizer, device)
        val_metrics = validate(model, val_loader, criterion, device)

        current_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_metrics['iou'])

        # Loss
        writer.add_scalars('Loss', {
            'train': train_metrics['loss'],
            'val': val_metrics['loss']
        }, epoch)

        # IoU
        writer.add_scalars('IoU', {
            'train': train_metrics['iou'],
            'val': val_metrics['iou']
        }, epoch)

        # Dice
        writer.add_scalars('Dice', {
            'train': train_metrics['dice'],
            'val': val_metrics['dice']
        }, epoch)

        # Precision
        writer.add_scalars('Precision', {
            'train': train_metrics['precision'],
            'val': val_metrics['precision']
        }, epoch)

        # Recall
        writer.add_scalars('Recall', {
            'train': train_metrics['recall'],
            'val': val_metrics['recall']
        }, epoch)

        # F1-Score
        writer.add_scalars('F1-Score', {
            'train': train_metrics['f1_score'],
            'val': val_metrics['f1_score']
        }, epoch)

        # Learning Rate
        writer.add_scalar('Learning_Rate', current_lr, epoch)

        # # Accuracy (tylko validation)
        # writer.add_scalar('Validation/Accuracy', val_metrics['accuracy'], epoch)

        # SAVE BEST MODEL
        if val_metrics['iou'] > best_val_iou:
            best_val_iou = val_metrics['iou']
            best_epoch = epoch
            patience_counter = 0

            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_iou': best_val_iou,
                'val_metrics': val_metrics,
                'train_metrics': train_metrics,
            }

            torch.save(checkpoint, f'best_model_iou_{best_val_iou:.4f}.pth')
        else:
            patience_counter += 1

        # ========================================
        # EARLY STOPPING
        # ========================================
        if patience_counter >= EARLY_STOPPING_PATIENCE:
            print(f"\n  Early stopping triggered!")
            print(f"   No improvement for {EARLY_STOPPING_PATIENCE} epochs")
            print(f"   Best IoU: {best_val_iou:.4f} at epoch {best_epoch + 1}")
            break

    
        val_images, val_masks = next(iter(val_loader))
        val_images = val_images.to(device)
        val_masks = val_masks.to(device)

        with torch.no_grad():
            val_outputs = model(val_images)
            val_preds = torch.sigmoid(val_outputs)

        # Weź pierwsze 4 obrazy
        writer.add_images('Images/Input', val_images[:4], epoch)
        writer.add_images('Images/Ground_Truth', val_masks[:4], epoch)
        writer.add_images('Images/Prediction', val_preds[:4], epoch)


    writer.close()
    print("Training done.")


#     # Load best model
    checkpoint = torch.load(f'best_model_iou_{best_val_iou:.4f}.pth', weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])

    test_metrics = validate(model, test_loader, criterion, device)
#      # Zapisz hyperparametry vs metryki (do porównania w TensorBoard)
#     writer.add_hparams(
#         {
#             'lr': LEARNING_RATE,
#             'batch_size': BATCH_SIZE,
#             'weight_decay': WEIGHT_DECAY,
#             'scheduler_patience': SCHEDULER_PATIENCE,
#         },
#         {
#             'hparam/test_iou': test_metrics['iou'],
#             'hparam/test_dice': test_metrics['dice'],
#             'hparam/test_f1': test_metrics['f1_score'],
#         }
#     )

#     writer.add_text('Final_Test_Metrics', f"""
    # Test Set Results (Best Model from Epoch {best_epoch + 1})

# -       IoU: {test_metrics['iou']:.4f}
# -      Dice: {test_metrics['dice']:.4f}
# -    Recall: {test_metrics['recall']:.4f}
# - Precision: {test_metrics['precision']:.4f}
# -  F1-Score: {test_metrics['f1_score']:.4f}
# Best Validation IoU: {best_val_iou:.4f}
#     """)

#     # Dodaj finalne metryki jako skalary
#     writer.add_scalar('Final/Test_IoU', test_metrics['iou'], 0)
#     writer.add_scalar('Final/Test_Dice', test_metrics['dice'], 0)
#     writer.add_scalar('Final/Test_Recall', test_metrics['recall'], 0)
#     writer.add_scalar('Final/Test_Precision', test_metrics['precision'], 0)
#     writer.add_scalar('Final/Test_F1', test_metrics['f1_score'], 0)



    print(f"\nTest Results:")
    print(f"   IoU:       {test_metrics['iou']:.4f}")
    print(f"   Dice:      {test_metrics['dice']:.4f}")
    print(f"   Recall:    {test_metrics['recall']:.4f}")
    print(f"   Precision: {test_metrics['precision']:.4f}")
    print(f"   F1-Score:  {test_metrics['f1_score']:.4f}")


    return 0

In [2]:
main()

NameError: name 'datetime' is not defined

In [66]:
import gc

In [67]:
print(f"Memory allocated before clearing cache: {torch.cuda.memory_allocated()} bytes")

# Manually invoke garbage collection

# Clear GPU cache

torch.cuda.empty_cache()
gc.collect()
print(f"Memory allocated after clearing cache: {torch.cuda.memory_allocated()} bytes")
torch.cuda.empty_cache()

Memory allocated before clearing cache: 417080320 bytes
Memory allocated after clearing cache: 417080320 bytes


In [68]:
gc.collect()
torch.cuda.empty_cache()

